# Lab 6 Function Calling and Tools

## 1. Function Call

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # 敏感信息不能明文展示
api_key = os.getenv('OPENAI_SECRET')

client = OpenAI(api_key=api_key)

## 1.1 Schema

In [ ]:
# Display the image
file_path = "data/function-calling-diagram-steps.png"
from IPython.display import Image
Image(filename=file_path)

## 1.2 工具选择

In [4]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

input_messages = [{"role": "user", "content": "北京天气怎么样？"}]


response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools
)



In [ ]:
tool_call = response.output[0]

print(f"你需要执行函数 {tool_call.name} ，入参是 {tool_call.arguments}")


### ? **思考**

大模型在这里做了几步推理？

## 1.2 本地工具执行

In [12]:
def get_weather(latitude:float, longitude:float) -> float:
    # mock 
    print(f"latitude: {latitude}, longitude: {longitude}")
    return 14.0

In [ ]:
import json
args = json.loads(tool_call.arguments)

# 先让我们手动执行
result = get_weather(args["latitude"], args["longitude"])
print(result)

特殊的python语法糖，用`**`传入所有参数的键值对

In [ ]:
result = get_weather(**args)
print(result)

`eval`函数将字符串定义的操作符计算

In [ ]:
eval("3*4")

In [ ]:
# 利用eval这个函数执行一个字符串
exec_command = f"{tool_call.name}(**{args})"  # NOTE:在这里用 f"{tool_call.name}(**{tool_call.arguments})"也可以
result = eval(exec_command)
print(f"execute {exec_command} and get result {result}")

## 1.3 回传(Callback)

通过call id来实现同一个会话的上下文跟踪

In [ ]:
input_messages.append(tool_call)  # append model's function call message
input_messages.append({                               # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

让我们来看一下现在的对话历史是什么了

In [ ]:
for i, item in enumerate(input_messages):
    print(f"msg {i+1}: {item}")

# 1.4 Tool定义

将工具函数的信息给大模型。 最重要的三个字段

- name: 函数名称
- description: 函数功能定义，明确、详细
- parameters: 以json格式定义的函数入参，这样可以通过`**`魔法糖传参

```
{
    "type": "function",
    "function": {
        "name": < 函数名称 >,
        "description": < 函数描述 >,
        "parameters": {
            "type": "object",
            "properties": {
                <入参 1> : {
                    "type": <入参1 类型>,
                    "description": <入参1 描述>
                },
                "<入参 2>": {
                    "type": "string",
                    "enum": [
                        "celsius",
                        "fahrenheit"
                    ], 
                    "description": <入参2 描述>
                }
            },
            "required": [
                 <入参 1>
            ],
            "additionalProperties": false
        },
        "strict": true
    }
}
```

### ？ 思考

上面的函数定义中：
- 总共定义了几个参数？
- 有哪个参数是强制传入的，哪个参数可选

# 2. 多函数选择

## 2.1 定义邮件发送工具

In [2]:
email_tool_config = {
    "type": "function",
    "name": "send_email",
    "description": "Send an email to the given recipient.",
    "parameters": {
        "type": "object",
        "properties": {
            "recipient": {"type": "string"},
            "subject": {"type": "string"},
            "body": {"type": "string"}
        },
        "required": ["recipient", "subject", "body"],
        "additionalProperties": False
    },
    "strict": True
}

In [3]:
weather_tool_config = {
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}

In [4]:
new_tools = [weather_tool_config, email_tool_config]

In [ ]:
for i, tool in enumerate(new_tools):
    print(f"第 {i+1} 个工具")
    print(tool)

## 2.2 多步执行

In [13]:
new_input_messages = [{"role": "user", "content": "查下北京的天气，然后给foo@bar.com发一封邮件告知内容"}]


response = client.responses.create(
    model="gpt-4.1",
    input=new_input_messages,
    tools=new_tools
)


In [ ]:
print(f"返回 {len(response.output)}个结果")
for step in response.output:
    print(f"类型是 {step.type}")
    print(step)

In [ ]:
for step in response.output:
    new_input_messages.append(step)
    result = eval(f"{step.name}(**{step.arguments})")
    new_input_messages.append({                               # append result message
        "type": "function_call_output",
        "call_id": step.call_id,
        "output": str(result)
    })

In [15]:
response_2 = client.responses.create(
    model="gpt-4.1",
    input=new_input_messages,
    tools=new_tools,
)

In [ ]:
print(f"返回{len(response_2.output)}个结果")
for step in response_2.output:
    print(f"类型是 {step.type}")
    print(step)

In [ ]:
print(f"id是 {response.output[0].id}, call_id是 {response.output[0].call_id}")
print(f"id是 {response_2.output[0].id}, call_id是 {response_2.output[0].call_id}")

### ? 思考

你观察到什么？LLM是怎么管理上下文的

In [18]:
def send_email(recipient:str, subject:str, body:str)->str:
    return "execution failed. recipient is not valid"

In [19]:
for step in response_2.output:
    new_input_messages.append(step)
    result = eval(f"{step.name}(**{step.arguments})")
    new_input_messages.append({                               # append result message
        "type": "function_call_output",
        "call_id": step.call_id,
        "output": str(result)
    })

In [20]:
response_3 = client.responses.create(
    model="gpt-4.1",
    input=new_input_messages,
    tools=new_tools,
)

In [ ]:
print(f"返回{len(response_3.output)}个结果")
for step in response_3.output:
    print(f"类型是 {step.type}")
    print(step)

In [ ]:
print(response_3.output_text)


# 3. **M**odel **C**ontext **P**rotocal

有一些通用工具，不要重复造轮子，能不能提供一共公用的工具定义函数？

这就是MCP

OpenAI API对于MCP的支持 kinda sucks（as of 25年6月），而Claude在国内非常难获得，所以MCP不作代码演示。

好在Cursor对于MCP的支持很好，所以我可以在Cursor中给大家演示

下面从[MCP图鉴](https://mcp.so/)里 引用[HowToCook](https://mcp.so/server/howtocook-mcp/worryzyy?tab=tools)这个MCP，解决“今晚吃什么”的世纪难题
```
npm install -g howtocook-mcp
```

然后去Cursor Settings -> MCP Tools，粘贴以下代码

```
{
  "mcpServers": {
    "howtocook-mcp": {
      "command": "npx",
      "args": [
        "-y",
        "howtocook-mcp"
      ]
    }
  }
}
```

然后打开大模型聊天页面，输入世纪难题

# 4. 工程化

虽然上面我们已经实现了一个ReAct Agent，但是我们现在的定义方式不利于进一步扩展我们的服务。

因此，我们需要对于Agent进行一些“抽象”和“封装”。 事实上有大量的Agent框架可以选择，我们完全不必要自己造轮子，但是为了能够让大家体验一下“手搓”Agent，我在这里提供一个代码框架

```
agent_tutorial
| - __init__.py  # 函数包的标识
| - tools.py           # 配置函数
| - tool_config.py  # 配置openai的tools config
| - agent.py        # Agent业务逻辑 
```